In [ ]:
import random
import pandas as pd


In [ ]:
import pandas as pd
class FLIGHT_LEG:
    def __init__(self, origin, destination, dep_time,arr_time,flight_no,index,duration,day):
        self.origin = origin
        self.destination = destination
        self.dep_time=dep_time
        self.arr_time=arr_time
        self.flight_no=flight_no
        self.visited=False
        self.index=index
        self.duration=duration
        self.day=day
    def disp_flight_details(self):
        print("Origin="+self.origin+",Destination="+self.destination+",DepTime="+str(self.dep_time)+",ArrTime="+str(self.arr_time)+",FlightNo"+str(self.flight_no)+ ",Visited="+str(self.visited)+",Index="+str(self.index)+",Duration="+str(self.duration)+",Day="+str(self.day))
def time_converter(hr,minute,day):
    total_time=hr*60+minute + (24*60)*day
    return total_time

def time_converter_for_duration(t):
    total_time=t*60
    return total_time


def create_legs_object(df,days=2):
    origin='a'
    destination='a'
    dep_time_char_hour='a'
    dep_time_char_min='a'
    arr_time_char_hour='a'
    arr_time_char_min='a'
    dep_time=0
    arr_time=0
    flight_no="a"
    legs=[]
    count=0
    no_of_days=days
    for d in range(no_of_days):
        for i in range(len(df)) :
            origin=df.loc[i, "Origin"]
            destination=df.loc[i, "Destination"]
            dep_time_char_hour=df.loc[i, "DepHour"]
            dep_time_char_min=df.loc[i, "DepMin"]
            arr_time_char_hour=df.loc[i,"ArrHour"]
            arr_time_char_min=df.loc[i, "ArrMin"]
            dep_time=time_converter(int(dep_time_char_hour),int(dep_time_char_min),d)
            arr_time=time_converter(int(arr_time_char_hour),int(arr_time_char_min),d)
            flight_no=df.loc[i,'FlightNum']
            duration=df.loc[i,'Duration'] #time_converter_for_duration(df.loc[i,'Duration'])
            legs.append(FLIGHT_LEG(origin,destination,dep_time,arr_time,int(flight_no),count,duration,d))
            count+=1
    return legs
def return_all_airports_dep_arr(legs):
    """
    All_airports is just a set of union of all arriving and departing airports
    airport_dep_flights is a dictionary of the following format : {'airport_code':[all departing flights from that airport code]}
    Similarly airport_arr_flights
    """
    all_airports=[]
    for item in legs:
        if item.origin not in all_airports:
            all_airports.append(item.origin)
        if item.destination not in all_airports:
            all_airports.append(item.destination)

    airport_dep_flights={}
    airport_arr_flights={}

    for airport in all_airports:
        airport_dep_flights[airport]=[]
        for item in legs:
            if(item.origin==airport):
                airport_dep_flights[airport].append(item)

    for airport in all_airports:
        airport_arr_flights[airport]=[]
        for item in legs:
            if(item.destination==airport):
                airport_arr_flights[airport].append(item)
    return all_airports,airport_arr_flights,airport_dep_flights
def count_feasible_connections(adjacency_matrix):
    """
    This is used as a counter/verifier to check how many feasible connections are present in the adjacency_matrix_of_connection
    If adjacency_matrix[i][j]==1, then the count is updated.
    """
    count=0
    for i in range(len(adjacency_matrix)):
        for j in range(len(adjacency_matrix[i])):
            if adjacency_matrix[i][j]==1:
                count+=1
    return count


def create_adjacency_matrix(legs):
 
    adjacency_matrix_of_connections=[[0]*len(legs) for i in range(len(legs))]
    for i in range(len(legs)):
        for j in range(len(legs)):
             if legs[i].destination==legs[j].origin and legs[j].dep_time-legs[i].arr_time>=20 and legs[j].dep_time-legs[i].arr_time<=180: 
                adjacency_matrix_of_connections[i][j]=1
    #print(count_feasible_connections(adjacency_matrix_of_connections))
    return adjacency_matrix_of_connections


from collections import defaultdict


class Graph:

    def __init__(self, vertices):
        # No. of vertices
        self.V = vertices

        # default dictionary to store graph
        self.graph = defaultdict(list)
    # function to add an edge to graph
    def addEdge(self, u, v):
        self.graph[u].append(v)

    '''A recursive function to print all paths from 'u' to 'd'.
    visited[] keeps track of vertices in current path.
    path[] stores actual vertices and path_index is current
    index in path[]'''



    def printAllPathsUtil(self, u, d, visited, path,dg):

        # Mark the current node as visited and store in path
        visited[u]= True
        path.append(u)

        # If current vertex is same as destination, then print
        # current path[]
        if u == d:
            dg.duty_validity_checker_and_writer(path)
        else:
            # If current vertex is not destination
            # Recur for all the vertices adjacent to this vertex
            for i in self.graph[u]:
                if visited[i]== False:
                    self.printAllPathsUtil(i, d, visited, path,dg)

        # Remove current vertex from path[] and mark it as unvisited
        path.pop()
        visited[u]= False


    # Prints all paths from 's' to 'd'
    def printAllPaths(self, s, d,dg):

        # Mark all the vertices as not visited
        visited =[False]*(self.V)

        # Create an array to store paths
        path = []

        # Call the recursive helper function to print all paths
        self.printAllPathsUtil(s, d, visited, path,dg)

class Duty:
    """
    Each duty period is essentially a Duty class
    This sets of flight legs that comprise a DP are stored in the self.duty attribute
    All other attributes like airtime etc are calculated using this self.duty attribute
    """
    def __init__(self,duty):
        self.duty=duty
        self.airtime,self.total_duration=self.calculate_airtime_and_total_duration(self.duty)
        self.duty_cost=self.calculate_duty_cost(self.duty)
        self.flight_indices=self.calculate_flight_indices(self.duty)
        self.duty_reduced_cost=self.calculate_duty_cost(self.duty)
    def calculate_duty_cost(self,duty):
        """
        The duty cost is obtained from Cynthia Barnhart s' textbook definition. 
        duty_cost=max(5/8* total_duration,flying_time, mintime=8hours)
        """
        elapse_cost=(5/8)*self.total_duration
        fly_cost=self.airtime
        min_guar=8
        duty_cost=max(elapse_cost,fly_cost,min_guar)
        return duty_cost
        
    def calculate_airtime_and_total_duration(self,duty):
        """
        Given a duty period, this calculates the total flying time(airtime) and the total duration(inclusive of wait time
        between flights) and returns it
        """
        airtime=0
        total_duration=0
        if len(duty)==1:
            airtime=duty[0].duration
            total_duration=0
        else:
            for i in range(len(duty)-1):
                airtime+=duty[i].duration
                total_duration+=(duty[i].duration+(duty[i+1].dep_time-duty[i].arr_time))
            airtime+=duty[-1].duration
            total_duration+=duty[-1].duration
        return airtime,total_duration
    def disp_duty_details(self):
        for item in self.duty:
            item.disp_flight_details()
    def calculate_flight_indices(self,duty):
        """
        Takes all flight indices related to a duty period and combines it into a list.
        """
        flight_indices=[]
        for leg in duty:
            flight_indices.append(leg.index)
        return flight_indices
            
            
            
        
            
        
class DutyGenerator:
    def __init__(self,adj_matrix,duty_hour_limit=8*60, filename="2DayRoutingUltimateAir-190.txt",legs=[]):
        self.adjacency_matrix=adj_matrix
        self.max_duty_hour_limit=duty_hour_limit
        self.filename=filename
        self.duty_list=[]
        self.legs_count=len(self.adjacency_matrix)
        self.graph=Graph(self.legs_count)
        self.total_duty_count=0
        self.feasible_connections=count_feasible_connections(self.adjacency_matrix)
        self.legs=legs

    def create_graph(self):
        g = self.graph
        for i in range(self.legs_count):
            for j in range(self.legs_count):
                if(self.adjacency_matrix[i][j]==1):
                    g.addEdge(i,j)
    def duty_validity_checker_and_writer(self,path):
        """
        Any given duty has to satisfy the valid legal duty constraints/rules.
        If the duty period is under the max_duty_hour_limit->then its added into the duty list.
        """
        current_s=""
        total_time=0
        temp_duty=[]
        #print(path)
        for element in path:
            #current_s+=str(legs[element].flight_no)+"-" + str(legs[element].origin)+"-"+str(legs[element].destination) +"-"+str(legs[element].day)+" "
            total_time+=self.legs[element].duration
            temp_duty.append(self.legs[element])

        if total_time<= self.max_duty_hour_limit:
            self.duty_list.append(Duty(temp_duty.copy()))
            self.total_duty_count+=1


In [ ]:
def create_leg_to_index_mapper(legs):
    """
    A leg to index mapper is basically of the below format:
    leg_to_index_mapper={leg.index:i}
    index_to_leg_mapper={i:leg.index}
    Both of these mappers are sometimes used, when the leg subset with which we are working is not the entire set.
    When its just a subset, then brute indexing might lead to a whole lot of problems.
    Mappers on the other hand can retain the original indexing. And hence these can avoid a lot of index out of range issues..etc
    """
    mapper={}
    for i in range(len(legs)):
        mapper[legs[i].index]=i
    return mapper

def create_index_to_leg_mapper(legs):
    mapper={}
    for i in range(len(legs)):
        mapper[i]=legs[i].index
    return mapper
        

In [ ]:
# count_feasible_connections(adjacency_matrix_of_connections)

In [ ]:
def get_all_departure_flights_and_organize(all_airports,airport_dep_flights):
    start_airports=[]
    end_airports=[]
    for airport in all_airports:
        start_airports.append(airport_dep_flights[airport])
        end_airports.append(airport_dep_flights[airport])
    return start_airports,end_airports

def generate_duties(all_airports,airport_dep_flights,adjacency_matrix_of_connections,leg_to_index_mapper,legs): 
    """
    This function basically creates a graph(with adjacency_matrix_of_connections). Then it performs DFS over the graph and identifies
    all valid duty periods.
    """
    start_airports,end_airports=get_all_departure_flights_and_organize(all_airports,airport_dep_flights)
    dg=DutyGenerator(adjacency_matrix_of_connections,8*60,"UltimateAir2DayDutyPeriodList-SmallSchedule-10.txt",legs) 
    dg.create_graph()
    for start_airport in start_airports:
        for s_flight in start_airport:
            s=leg_to_index_mapper[s_flight.index]

            for end_airport in end_airports:
                for e_flight in end_airport:
                    d=leg_to_index_mapper[e_flight.index]
                    dg.graph.printAllPaths(s,d,dg)
    return dg

                     

In [ ]:
def coverage_checker(pg,leg_to_index_mapper,legs):
    """
    Coverage Matrix is a matrix, where the rows are indexed by flights_indices and the columns are indexed
    by pairings. Each pairing consists of many flight indices. Therefore, coverage_matrix[flightIndex][pairingIndex]=1
    If the pairing consists of that flight index, else 0
    We also return the uncovered_flight_indices if the coverage_matrix does not cover all flights. 
    """
    coverage_matrix=[]
    flight_coverage_checker=[]
    for i in range(len(leg_to_index_mapper.values())):
        temp=[]
        flight_coverage_checker.append(0)
        for j in range(len(pg.pairings)):
            temp.append(0)
        coverage_matrix.append(temp)
    for j in range(len(pg.pairings)):
        for indices in pg.pairings[j].flight_indices:
            flight_coverage_checker[leg_to_index_mapper[indices]]+=1
            coverage_matrix[leg_to_index_mapper[indices]][j]=1

    # Checking if all flights have been covered at least under 1 pairing
    flagger=True
    non_covered_airports=[]
    uncovered_flight_indices=[]
    for i in range(len(flight_coverage_checker)):
        if flight_coverage_checker[i]==0:
            non_covered_airports.append(legs[i].origin)
            #non_covered_airports.append(legs[i].destination)
            non_covered_airports=list(set(non_covered_airports))
            if flagger!=False:
                flagger=False
            
            uncovered_flight_indices.append(i)
    non_covered_airports=list(set(non_covered_airports).difference(set(pg.crew_base)))
    if flagger==True:
        print("Yay! All Flights Covered")
        return (True,coverage_matrix)
    else:
        print("Oh No! Not all Flights Covered")
        #print("The non covered airports are",non_covered_airports)
        print("")
        return (False,coverage_matrix,uncovered_flight_indices,leg_to_index_mapper)
    
    


In [ ]:
def return_crew_bases_indices(crew_base,start):
    crew_base_start={}
    for i in range(len(crew_base)):
        crew_base_start[crew_base[i]]=(start+i,start+len(crew_base)+i)
    return crew_base_start

#Original Function defined by me

def duty_network(dg,crew_base,overnight_layover_constant=8*60):
    """
    duty_network is matrix, that consists of duties as nodes. The edges are present between duties,
    i.e duty_network[i][j] =1, only if the overnight layover time between 2 duties i and j is > 8 hours(or overnight_layover_constant)
   
    Once, the duty network is formed, any legal(valid) pairing, is formed between source and destination where:
    Source:is a duty period node that starts at a crew bases
    Destination:is a duty period node that ends at a crew base that started from source
    """
    duty_network=[]
    no_of_crew_bases=len(crew_base)
    crew_bases_indices=return_crew_bases_indices(crew_base,len(dg.duty_list))
    
    for i in range(len(dg.duty_list)+(no_of_crew_bases*2)):
        temp=[]
        for j in range(len(dg.duty_list)+(no_of_crew_bases*2)):
            temp.append(0)
        duty_network.append(temp)
        
    for i in range(len(dg.duty_list)):
        for j in range(len(dg.duty_list)):
            if i!=j:
                if (dg.duty_list[j].duty[0].dep_time-dg.duty_list[i].duty[-1].arr_time >= overnight_layover_constant) and (dg.duty_list[i].duty[-1].destination==dg.duty_list[j].duty[0].origin):
                    duty_network[i][j]=1
    #Connect Origin Airports to their appropriate crew start bases 
    for to_ in range(len(dg.duty_list)):
        start_flight=dg.duty_list[to_].duty[0].origin
        for keys,values in crew_bases_indices.items():
            crew_base_from_index=values[0]
            if keys==start_flight: #First flight of duty period s' origin has matched
                duty_network[crew_base_from_index][to_]=1
    #Connect Destination airports to their appropriate crew bases
    for from_ in range(len(dg.duty_list)):
        end_flight=dg.duty_list[from_].duty[-1].destination
        for keys,values in crew_bases_indices.items():
            crew_base_to_index=values[1]
            if keys==end_flight: #Last flight of duty period s' destination has matched
                duty_network[from_][crew_base_to_index]=1
                
    return duty_network,crew_bases_indices
        
            
            
# import numpy as np
# from scipy.sparse import lil_matrix

# def duty_network(dg, crew_base, overnight_layover_constant=8 * 60):
#     no_of_crew_bases = len(crew_base)
#     crew_bases_indices = return_crew_bases_indices(crew_base, len(dg.duty_list))

#     # Create a sparse matrix using lil_matrix (List of List format)
#     duty_network = lil_matrix((len(dg.duty_list) + (no_of_crew_bases * 2), len(dg.duty_list) + (no_of_crew_bases * 2)), dtype=int)

#     for i in range(len(dg.duty_list)):
#         #print(i)
#         for j in range(len(dg.duty_list)):
#             if i != j:

#                 if (dg.duty_list[j].duty[0].dep_time - dg.duty_list[i].duty[-1].arr_time >= overnight_layover_constant) and (
#                         dg.duty_list[i].duty[-1].destination == dg.duty_list[j].duty[0].origin):
#                     duty_network[i, j] = 1

#     # Connect Origin Airports to their appropriate crew start bases
#     for to_ in range(len(dg.duty_list)):
#         start_flight = dg.duty_list[to_].duty[0].origin
#         for keys, values in crew_bases_indices.items():
#             crew_base_from_index = values[0]
#             if keys == start_flight:
#                 duty_network[crew_base_from_index, to_] = 1

#     # Connect Destination airports to their appropriate crew bases
#     for from_ in range(len(dg.duty_list)):
#         end_flight = dg.duty_list[from_].duty[-1].destination
#         for keys, values in crew_bases_indices.items():
#             crew_base_to_index = values[1]
#             if keys == end_flight:
#                 duty_network[from_, crew_base_to_index] = 1

#     return duty_network, crew_bases_indices

# import multiprocessing

# def calculate_matrix_entry(args):
#     i, j, dg, overnight_layover_constant = args
#     print(i,j)
#     if i != j:
#         if (dg.duty_list[j].duty[0].dep_time - dg.duty_list[i].duty[-1].arr_time >= overnight_layover_constant) and (
#                 dg.duty_list[i].duty[-1].destination == dg.duty_list[j].duty[0].origin):
#             return (i, j, 1)
#     return (i, j, 0)

# def duty_network(dg, crew_base, overnight_layover_constant=8 * 60):
#     no_of_crew_bases = len(crew_base)
#     crew_bases_indices = return_crew_bases_indices(crew_base, len(dg.duty_list))

#     # Create a pool of worker processes
#     num_processes = multiprocessing.cpu_count()
#     pool = multiprocessing.Pool(num_processes)

#     args_list = [(i, j, dg, overnight_layover_constant) for i in range(len(dg.duty_list)) for j in range(len(dg.duty_list))]

#     # Parallel computation of matrix entries
#     result = pool.map(calculate_matrix_entry, args_list)

#     # Close the pool to release resources
#     pool.close()
#     pool.join()

#     # Create a sparse matrix from the results
#     duty_network = lil_matrix((len(dg.duty_list) + (no_of_crew_bases * 2), len(dg.duty_list) + (no_of_crew_bases * 2)), dtype=int)
    
#     for i, j, value in result:
#         duty_network[i, j] = value

#     # Connect Origin Airports to their appropriate crew start bases
#     # Connect Destination airports to their appropriate crew bases (as in your original code)

#     return duty_network, crew_bases_indices




In [ ]:
def NXGRAPH_VISUALIZER(adj_matrix):
    """
    This is to visualise the duty network or any adjacency matrix given as input.
    The 2 adjancency matrices that we commonly use are: adjacency_matrix_of_connections 
    and duty_network
    """
    import networkx as nx
    import matplotlib.pyplot as plt
    G=nx.DiGraph()
    for i in range(len(adj_matrix)):
        G.add_node(i)
    for i in range(len(adj_matrix)):
        for j in range(len(adj_matrix)):
            if i!=j:
                if adj_matrix[i][j]==1:
                    G.add_edge(i,j)
                    
    nx.draw(G)
    return G
    
#G=NXGRAPH_VISUALIZER(duty_network)

In [ ]:
class Pairing:
    def form_pairing(self,pairing):
        self.pairing=pairing
        self.duration_of_pairing=self.calculate_duration_of_pairing(self.pairing)
        self.flight_indices=self.calculate_flight_indices(self.pairing)
        self.TAFB=self.calculate_TAFB(self.pairing)
        self.no_of_duty_periods=self.calculate_no_of_duty_periods(self.pairing)
        self.total_duty_cost=self.calculate_total_duty_cost(self.pairing)
        self.pairing_cost=self.calculate_pairing_cost(self.pairing,self.TAFB,self.no_of_duty_periods,self.total_duty_cost)
        self.total_flying_time=self.calculate_total_flying_time(self.pairing)
        self.total_elapse_time=self.calculate_total_elapse_time(self.pairing)
        self.pairing_reduced_cost=self.calculate_reduced_cost(self.pairing)
        
    def __init__(self,pairing=[]):
        self.pairing=[]
        self.duration_of_pairing=0
        self.flight_indices=[]
        self.TAFB=0
        self.no_of_duty_periods=0
        self.total_duty_cost=0
        self.pairing_cost=0
        self.total_flying_time=0
        self.total_elapse_time=0
        self.pairing_reduced_cost=0
        if len(pairing)!=0:
            self.form_pairing(pairing)
        
    def calculate_duration_of_pairing(self,pairing):
        days=int(((pairing[-1].duty[-1].arr_time-pairing[0].duty[0].dep_time)/60)/24)
        return days
        
    def calculate_flight_indices(self,pairing):
        flight_indices=[]
        for item in pairing:
            duty=item.duty
            for flight in duty:
                flight_indices.append(flight.index)
        return flight_indices
                
    def calculate_TAFB(self,pairing):
        TAFB=int(((pairing[-1].duty[-1].arr_time-pairing[0].duty[0].dep_time)/60)/24)
        return TAFB
    
    def calculate_no_of_duty_periods(self,pairing):
        return len(pairing)
    
    def calculate_total_duty_cost(self,pairing):
        cost=0
        for duty in pairing:
            cost+=duty.duty_cost
        return cost
    
    def calculate_pairing_cost(self,pairing,TAFB,no_of_duty_periods,total_duty_cost):
        fp=10
        mg=10
        cost=max(fp*TAFB,mg*no_of_duty_periods,total_duty_cost)
        link_cost=0
        constant_overnight_layover_cost_per_hour=100
        if len(pairing)==1:
            link_cost=0
        else:
            for i in range(len(pairing)-1):
                duty1=pairing[i].duty
                duty2=pairing[i+1].duty
                overnight_layover_time=duty2[0].dep_time-duty1[-1].arr_time
                overnight_layover_time_cost=constant_overnight_layover_cost_per_hour*overnight_layover_time
                link_cost+=overnight_layover_time_cost
        total_pairing_cost=cost+link_cost
        return total_pairing_cost
    
    def calculate_total_flying_time(self,pairing):
        total_flying_time=0
        for item in pairing:
            total_flying_time+=item.airtime
        return total_flying_time
    
    def calculate_total_elapse_time(self,pairing):
        total_elapse_time=0
        if len(pairing)!=0:
            for i in range(len(pairing)-1):
                total_elapse_time+=pairing[i].total_duration
                total_elapse_time+=(pairing[i+1].duty[0].dep_time-pairing[i].duty[-1].arr_time) # add overnight layover time
            total_elapse_time+=pairing[-1].total_duration
        else:
            total_elapse_time+=pairing[0].total_duration
        return total_elapse_time
    
    def calculate_reduced_cost(self,pairing):
        p_reduced_cost=0
        for i in range(len(pairing)):
            p_reduced_cost+=pairing[i].duty_reduced_cost
        return p_reduced_cost
            
        
        
    def disp_pairing_details(self):
        s=""
        for item in self.pairing:
            for leg in item.duty:
                s+="("+"IND="+str(leg.index)+","+str(leg.flight_no)+","+str(leg.origin)+"="+str(leg.destination)+","+str(leg.day) +")"+ "->"
        print(s)
    def form_label_list(self):
        label_list=(self.TAFB,self.no_of_duty_periods,self.pairing_reduced_cost,self.pairing_cost,self.total_flying_time,self.total_elapse_time)
        #label_list=(self.pairing_reduced_cost,self.pairing_cost)
        #label_list=(self.TAFB,self.total_duty_cost,self.total_flying_time,self.no_of_duty_periods)
        #print(label_list)
        return label_list

In [ ]:
class node:
    def __init__(self,index,duty_index,typ,neighbors):
        """
        A node class is defined in order to make SPPRC more organized. Each node has index, duty_index, neighbors, type and labels 
        as properties.
        The typ has usually 3 options -> s, t, d  .. where s means start, t means destination, d means duty.
        |S| = |crew_bases| and |T| =|crew_bases|
        S_crew_base is connected to all duty periods starting with that specific crew_base
        All duty periods ending with that specific crew base in connected to T_crew_base
        Hence, a valid crew pairing is represented as a path from S_Crew_base to E_crew_base satisfying many rules
    
        """
        self.index=index
        self.duty_index=duty_index
        self.neighbors=neighbors
        self.typ=typ
        self.labels=[]

def feasibility_check(p,d,pairing_legality_params):
    """
    A duty(d) needs to be added to an existing(p). Then new_p is formed. This new_p is checked if it satisifes all rules
    and legalities(for eg: Time Away from Base, Total Flying time and Max no. of Duty Periods)
    Only if the new_p satisfies all the legality rules, only then an indicator True with the new pairing shall be returned
    Else, False shall be returned
    """
    new_p=p.pairing
    new_p.append(d)
    new_p=Pairing(new_p)
    #Feasibility checking !
    #print(new_p.form_label_list())
    if new_p.TAFB<=pairing_legality_params['TAFB'] and new_p.total_flying_time<=pairing_legality_params['MAXFLYINGTIME'] and new_p.no_of_duty_periods<=pairing_legality_params['MAXDUTYPERIODS']:
        return (True,new_p)
    else:
        return (False,[])
    
def comparer(tuple1,tuple2):
    """
    2 tuples that are N dimensional are compared.
    Say Ti=(di1,di2....,din)
    Say Tj=(dj1,dj2....,djn)
    Here Ti is the entering resource list or the new resource list that has to be compared against all existing resource lists
    which is Tj
    If any resource of Ti say di_index <= dj_index of Tj, then a True is appended, which essentially means Tj cannot dominate Ti
    However, if not even 1 resource Ti can dominate Tj then it means Tj entirely dominates Ti and therefore Ti need not be added
    to labels and can be ignored from future computations.
    """
    comparer=[]
    #print(tuple1,tuple2)
    for i in range(len(tuple1)):
        if tuple1[i]<=tuple2[i]:
            comparer.append(True)
            break
        else:
            comparer.append(False)
    if True in comparer:
        return True
    else:
        return False
def dominate(entering_resource_list,labels):
    """
    For each node, there exists many labels. The entering resource list has to be compared against all labels of a particular node
    If every label dominates the entering resource list, then it is ignored. However, if the entering resource list dominates
    atleast one label of all labels present in that node in atleast 1 resource aspect, then it cannot be ignored and has to be
    accounted for.
    The comparison/domination check between the resources of 2 lists is done by comparer() function[Note: the comparer function
    sends entering_resource_list,old_resource_list as parameters]
    """
    entering_resource_list=entering_resource_list.form_label_list()
    label_check=[]
    for label in labels:
        old_resource_list=label[1].form_label_list()
        if comparer(entering_resource_list,old_resource_list)==True:
            label_check.append(True)
            break
        else:
            label_check.append(False)
    if True in label_check or len(label_check)==0:
        return True
    else:
        return False


In [ ]:
def SPPRC(airport_letter_code,duty_network,crew_bases_indices,s_nodes_dict,t_nodes_dict,dg):
    """
    This is the SPPRC Function code. The SPPRC function is iterated through each crew base in order to generate paths.
    """
    node_dict={}
    for i in range(len(duty_network)):
        neighbors=[]
        for j in range(len(duty_network[i])):
            if duty_network[i][j]==1:
                neighbors.append(j)
        duty_index=i
        if i <len(dg.duty_list):
            typ='d'
        else:
            if i in s_nodes_dict.values():
                typ='s'
            elif i in t_nodes_dict.values():
                typ='t'

        node_dict[i]=node(i,duty_index,typ,neighbors)
    start=crew_bases_indices[airport_letter_code][0]
    U=[[start,Pairing(),start]]
    pairing_legality_params={'TAFB':5,'MAXFLYINGTIME':50*60,'MAXDUTYPERIODS':8}
    stopping_criterion_pairings_count=10*100
    ct=0
    end=crew_bases_indices[airport_letter_code][1]
    while len(U) !=0:
        #SPPRC Limiting/Stopping Criterion to Control SPPRC Flow
        if len(node_dict[end].labels) >= stopping_criterion_pairings_count:
            return node_dict[end].labels
        else:
            current_item=U.pop(0)
            current_node=current_item[0]
            current_pairing=current_item[1]
            if dominate(current_pairing,node_dict[current_node].labels)==True:
                node_dict[current_node].labels.append(current_item)
                for extension in node_dict[current_node].neighbors:
                    if node_dict[extension].typ!='t':
                        f=feasibility_check(Pairing(current_pairing.pairing.copy()),dg.duty_list[node_dict[extension].duty_index],pairing_legality_params)
                        if f[0]==True:
                            U.append([extension,f[1],current_node])
                    elif node_dict[extension].typ=='t':
                        U.append([extension,current_pairing,current_node])
    end=crew_bases_indices[airport_letter_code][1]
    return node_dict[end].labels

def calculate_negative_reduced_cost(pairing,dual_values):
    dual_costs=0
    for flight_index in pairing.flight_indices:
        dual_costs+=dual_values[flight_index]
    
    if pairing.pairing_cost-dual_costs <=0:
        print(pairing.pairing_cost-dual_costs,"BRO")
        return True
    else:
        return False
def process_labels_and_collect_pairings(PG,labels,dual_values):
    for label in labels:
        checker=False
        for pairing in PG:
            if sorted(pairing.flight_indices)==sorted(label[1].flight_indices):
                checker=True
        if checker==False: 
            if calculate_negative_reduced_cost(label[1],dual_values)==True:
                PG.append(label[1])
    return PG

def form_duty_network(dg,crew_base):
    """
    This is just a helper function for duty_network()
    It returns the dutyNetwork, crew_bases_indices, s_nodes_dict, t_nodes_dict.
    The crew_bases_indices is a dictionary that has the format: {"airport_code":[start_node_index,end_node_index]}
    start_node_index is basically the index of the node in the duty_network that has the index responsible for itself connecting to all 
    duty periods starting with that crew base
    Same goes with the end_node_index->responsible for all duty periods ending at a particular crew base to connect itself with the appropriate
    corresponding end_node crew base
    Eg:The start_node corres to JFK is connected to  All DPs that start with JFK as its crew base.
    Similarly, all DPs ending with JFK are connected to end_node corres to JFK.
    s_nodes_dict contains the foll format {"airport_code":start_index}
    t_nodes_dict contains the foll format {"airport_code":end_index}
    
    """
    dutyNetwork,crew_bases_indices=duty_network(dg,crew_base)
    s_nodes_dict={}
    t_nodes_dict={}
    for k,v in crew_bases_indices.items():
        s_nodes_dict[k]=v[0]
        t_nodes_dict[k]=v[1]
    return dutyNetwork,crew_bases_indices,s_nodes_dict,t_nodes_dict

In [ ]:
class PairingsGenerator:
    def __init__(self,pairings=[]):
        self.pairings=[]
        self.overnight_rest=8*60
        #self.crew_base=['JFK','ATL','LAX','SFO','ORD','BOS','MIA']
        self.crew_base=['DEL', 'BLR', 'BOM', 'HYD', 'CCU', 'MAA', 'AMD', 'PNQ', 'GAU', 'LKO', 'VNS', 'BBI', 'COK', 'PAT', 'GOI', 'JAI', 'SXR', 'IXE', 'IMF']
        if len(pairings)!=0:
            self.form_pair_gen(pairings)
    def form_pair_gen(self,pairings):
        self.pairings=pairings
        self.overnight_rest=8*60
        #self.crew_base=['JFK','ATL','LAX','SFO','ORD','BOS','MIA']
        self.crew_base=['DEL', 'BLR', 'BOM', 'HYD', 'CCU', 'MAA', 'AMD', 'PNQ', 'GAU', 'LKO', 'VNS', 'BBI', 'COK', 'PAT', 'GOI', 'JAI', 'SXR', 'IXE', 'IMF']
        
    def generate_1_day_pairings(self,duty_list):
        for i in range(len(duty_list)):
            if duty_list[i].duty[0].origin in self.crew_base and duty_list[i].duty[-1].destination in self.crew_base and duty_list[i].duty[0].origin==duty_list[i].duty[-1].destination:
                self.pairings.append(Pairing([duty_list[i]]))
                             
    def generate_2_day_pairings(self,duty_list):
        for i in range(len(duty_list)):
            for j in range(len(duty_list)):
                if i!=j:
                    if duty_list[i].duty[-1].destination==duty_list[j].duty[0].origin:
                        if duty_list[i].duty[0].origin in self.crew_base and duty_list[j].duty[-1].destination in self.crew_base and duty_list[i].duty[0].origin==duty_list[j].duty[-1].destination:
                            if duty_list[j].duty[0].dep_time-duty_list[i].duty[-1].arr_time>=self.overnight_rest and duty_list[j].duty[0].dep_time>=duty_list[i].duty[-1].arr_time:
                                self.pairings.append(Pairing([duty_list[i],duty_list[j]]))
    def generate_short_round_trip_pairings(self,legs):
        """
        This function is mainly decide to try and quickly cover flights that can be covered by roundtrips
        It takes a start flight from a crew base and then tries to find a return flight back to the same crew base.
        If a return flight is found. Then the pairing is terminated and no other flight is added.
        This is not optimal but it is designed to cover a large set of flights.
        Eg: MAA-TRZ and TRZ-MAA form a round trip pairings.
        """
        for crew_base in self.crew_base:
            start_airport=crew_base
            end_airport=crew_base
            for start_leg in legs:
                if start_leg.origin == start_airport:
                    start_leg_origin=start_leg.origin
                    start_leg_destination=start_leg.destination
                    for end_leg in legs:
                        temp_duty=[]
                        if end_leg.origin==start_leg_destination and end_leg.dep_time-start_leg.arr_time >=20 and end_leg.dep_time-start_leg.arr_time <=240 and end_leg.destination==end_airport:
                            temp_duty.append(start_leg)
                            temp_duty.append(end_leg)
                            duty=Duty(temp_duty)
                            self.pairings.append(Pairing([duty]))
            
                                                     

                                        

In [ ]:
def get_objective_function_coefficients(pg,revised=True):
    objective_function_coefficients=[]
    for pairing in pg.pairings:
        if pairing.duration_of_pairing==0:
            objective_function_coefficients.append(1)
        elif pairing.duration_of_pairing==1:
            objective_function_coefficients.append(3)
        elif pairing.duration_of_pairing==2:
            objective_function_coefficients.append(6)
        elif pairing.duration_of_pairing==3:
            objective_function_coefficients.append(9)
        elif pairing.duration_of_pairing==4:
            objective_function_coefficients.append(12)
    objective_function_coefficients_revised=[]
    for pairing in pg.pairings:
        objective_function_coefficients_revised.append(pairing.pairing_cost)
    if revised==True:
        return objective_function_coefficients_revised
    else:
        return objective_function_coefficients

def create_and_solve_model(pg,leg_to_index_mapper,legs,IP=False):
    coverage=coverage_checker(pg,leg_to_index_mapper,legs)
    if coverage[0]==False:
        print("Not Possible To Proceed Until all flights covered!")
        print("Ending .....")
        return
    coverage_matrix=coverage[1]
    objective_function_coefficients=get_objective_function_coefficients(pg)
    from docplex.mp.model import Model
    airline_cp_model = Model('Airline Crew Pairing Model')
    if IP==False:
        x = airline_cp_model.continuous_var_list(len(pg.pairings), name="x")
    else:
        x = airline_cp_model.binary_var_list(len(pg.pairings), name="x")
    #print(x,len(legs))
    for i in range(len(legs)):
        airline_cp_model.add_constraint(sum(coverage_matrix[i][j]*x[j] for j in range(len(pg.pairings))) >= 1, 'airline')
    obj_fn=sum(objective_function_coefficients[i]*x[i] for i in range(len(pg.pairings)))
    airline_cp_model.set_objective("min",obj_fn)
# !!!!!!!!!! THE BELOW ARE IMPORTANT PRINT STATEMENTS DO NOT JUST DELETE THEM !!!!!!!!!!!!!!
#     print("**** \t AIRLINE CREW PAIRING MODEL INFORMATION \t ****")
#     airline_cp_model.print_information()
    print("**** \t SOLVING \t ****")
    sol=airline_cp_model.solve()
#     print("**** \t SOLVED \t ****")
#     print("**** !!!! \n\n\t SOLUTION DETAILS \t\n\n !!!! ****")
#     airline_cp_model.print_solution()
    return sol,airline_cp_model,x



def return_pairing_information(pg,airline_cp_model,x):
    pairing_solution_indices=[]
    for i in range(len(pg.pairings)):
        if airline_cp_model.solution.get_values(x)[i]==1:
            pairing_solution_indices.append(i)
    return pairing_solution_indices

def print_pairing_information(pg,pairing_solution_indices):
    for index in pairing_solution_indices:
        print("PAIRINGINDEX=",index,"---->",end='')
        pg.pairings[index].disp_pairing_details()
        print()
        
def return_dual_values(airline_cp_model):
    dual_values=airline_cp_model.dual_values(airline_cp_model.find_matching_linear_constraints('airline'))
    flight_duals={}
    for i in range(len(dual_values)):
        flight_duals[i]=dual_values[i]
    return flight_duals

def print_dual_values(dual_values,legs):
    for k,v in dual_values.items():
        print("FlightIndex=",k,"-->DualValue-->",dual_values[k],"||||| DETAILS=",end='')
        legs[k].disp_flight_details()
        print()
        
def modify_duty_costs(dual_values,dg):
    """
    This function is used to modify the duty values after the column generation iteration returns some pairings.
    Basically, it modifes the duty costs so that revised new pairings are obtained via SPPRC
    """
    for i in range(len(dg.duty_list)):
        current_duty_object=dg.duty_list[i]
        current_duty=current_duty_object.duty
        current_duty_flight_indices=current_duty_object.flight_indices
        for flight_index in current_duty_flight_indices:
            current_duty_object.duty_reduced_cost+=dual_values[flight_index]
            #current_duty_object.duty_cost+=dual_values[flight_index]
    return dg

def default_model_return_function(sol,airline_cp_model,x,pg):
    # !!!!!!!!!! THE BELOW ARE IMPORTANT PRINT STATEMENTS DO NOT JUST DELETE THEM !!!!!!!!!!!!!!
    if sol is None:
        print("\n\n\n\n !!!! Infeasible Oh No !!!!\n\n\n\n")
    else:
        print("\n\n\n\n!!!! Feasible Yes !!!! :) \n\n\n\n ")
        #print("\n\n\n\n!!!! PAIRING DETAILS !!!! \n\n\n\n ")
        pairing_solution_indices=return_pairing_information(pg,airline_cp_model,x)
        #print_pairing_information(pg,pairing_solution_indices)
        #print("\n\n\n\n Dual Information \n\n\n\n")
        dual_values=return_dual_values(airline_cp_model)
        #print_dual_values(dual_values,legs)
    return pairing_solution_indices,dual_values


In [ ]:
def create_flight_coverage_status(legs):
    """
    The flight_coverage_status initializes a dictionary to values as False for all legs present in legs.
    This status dict is used often to check if all flights have been covered or not
    or i.e if any flight is not yet covered.
    """
    flight_coverage_status={}
    for i in range(len(legs)):
        flight_coverage_status[legs[i].index]=False
    return flight_coverage_status
# def form_index_updated_pairing(entering_pairing,legs,index_to_leg_mapper,leg_to_index_mapper):
#     new_pairing=[]
#     pairing=entering_pairing.pairing
#     for i in range(len(pairing)):
#         current_duty=pairing[i].duty
#         temp_duty=[]
#         for j in range(len(current_duty)):
#             current_leg=current_duty[j]
#             revised_leg_index=leg_to_index_mapper[current_leg.index]
#             revised_leg=legs[revised_leg_index]
#             temp_duty.append(revised_leg)
#         temp_duty=Duty(temp_duty)
#         new_pairing.append(temp_duty)
#     new_pairing=Pairing(new_pairing)
#     return new_pairing
def update_pairing_set(PG,pg,pairing_solution_indices,legs,index_to_leg_mapper,leg_to_index_mapper):
    """
    The pg.pairings contains a new set of pairings that decide to enter the original list
    And the PG.pairings contains the old/original set of pairings. 
    if however, the sorted flight indices of a new pairing matches with the sorted flight indices of any
    existing pairing, it essentially means the pairing already exists, and therefore the new pairing can
    be ignored
    This function is mainly designed so that duplicates do not enter the original PG list.
    """
    for index in pairing_solution_indices:
        new_pairing=pg.pairings[index]
        checker=True
        for existing_pairing in PG:
            if sorted(existing_pairing.flight_indices)==sorted(new_pairing.flight_indices):
                checker=False
        if checker==True:
            PG.append(new_pairing)
    return PG

def update_flight_coverage_status(flight_coverage_status,flight_legs_chosen):
    """
    For each leg in flight_legs_chosen, the flight coverage status of that particular leg is updated from
    False to True.
    """
    for leg in flight_legs_chosen:
        flight_coverage_status[leg.index]=True
    return flight_coverage_status

In [ ]:
def init_update_flight_coverage_status(pairings,coverage_status):
    """
    Each pairing consists of many duties, and each duty consist of many flight indices/flights.
    In order to update the flight_coverage_status, one takes in a list of pairings, and for all those
    flights contained in the pairings, the indicator is updated to TRUE.    
    The constant updation of flight_coverage_status is needed since, it would then help us determine which
    flights are still uncovered and how to continue the divide and cover/any other heuristic. Also, if all the
    flights are True, then it means all flight have been covered by the set of pairings given and therefore, 
    we can stop Divide and Cover.
    """
    for pairing in pairings:
        for index in pairing.flight_indices:
            coverage_status[index]=True
    return coverage_status
def init_update_pairing_list(pairings,PG):
    """
    In every iteration of Divide and Cover/Divide and Cover Controller new pairings are generated. These need to be added
    to the global PG Set. This function helps in doing so. This function is mainly used in the Divide and Cover Controller code.
    """
    for pairing in pairings:
        PG.append(pairing)
    return PG
def update_flight_list(updated_flight_legs,legs,global_flight_legs_copy,flight_coverage_status,K,main_iter):
    """
    This takes in as input "updated_flight_legs, legs, global_flight_legs_copy, flight_coverage_status, K, main_iter"
    1. For all flights that are not covered(i.e flight_coverage_status[index]=False), it takes into consideration
    the Origin and Destination airport. Also these uncovered flights are added to updated_flight_legs
    2. Then, We form a set called uncov_airports
    3. For the airports, present in uncov_airports, we find out all flights that have either the origin or destination as 
    one of these airports
    Eg: Lets say IXZ was an uncovered airport, we add all flights originating and departing from this airport to updated_flight_legs
    4. While adding a flight to updated_flight_legs, we need to make sure that its not already there. Duplicates cause 
    huge problems while creating coverage matrices and then later on solving them.
    5. Return the updated_flight_legs

    """
    uncov_airports=[]
    cov_indices=[]
    upd_indices=[]
    for k,v in flight_coverage_status.items():
        if v==False:
            updated_flight_legs.append(legs[k])
            uncov_airports.append(legs[k].origin)
            uncov_airports.append(legs[k].destination)
            cov_indices.append(k)
            upd_indices.append(k)
    uncov_airports=list(set(uncov_airports))
    all_airports,airport_arr_flights,airport_dep_flights=return_all_airports_dep_arr(global_flight_legs_copy.copy())
    if len(updated_flight_legs)<K and main_iter%5==0:
        for airport in uncov_airports:
            for flight in airport_arr_flights[airport]:
                if flight.index not in cov_indices and flight.index not in upd_indices:
                    updated_flight_legs.append(flight)
                    upd_indices.append(flight.index)
            for flight in airport_dep_flights[airport]:
                if flight.index not in cov_indices and flight.index not in upd_indices:
                    updated_flight_legs.append(flight)
                    upd_indices.append(flight.index)
    return updated_flight_legs
def new_coverage_matrix_and_legs(coverage,flight_legs_chosen):
    """
    1. This takes in as input(coverage,flight_legs_chosen)
    2. Remember coverage has the following format-> (False, old_coverage_matrix, list of uncovered flight indices)
    3. Using the old_coverage_matrix, and uncovered_flight_index... we do the following
        3.1. We form an empty matrix called new_coverage_matrix. Note in the below code we have not started with any predefined size
        3.2. We then loop through the old_coverage_matrix s' rows
        3.3. If the index is in uncovered_flight_indices, then do not add the corresponding row into new_coverage_matrix
        3.4. Correspondingly, if the index is not in uncovered_flight_indices, add the corresponding row from old_coverage_matrix
        and the corresponding flight index into new_flight_legs
    4. This function s' main purpose is to prepare the new_coverage_matrix and the new_flight_legs_covered for purpose of IP CPLEX Optimization
    
    
    """
    uncovered_flight_indices=coverage[2]
    old_coverage_matrix=coverage[1].copy()
    new_coverage_matrix=[]
    new_flight_legs_covered=[]
    for i in range(len(old_coverage_matrix)):
        if i not in uncovered_flight_indices:
            new_coverage_matrix.append(old_coverage_matrix[i].copy())
            new_flight_legs_covered.append(flight_legs_chosen[i])
    return new_flight_legs_covered,new_coverage_matrix


def init_for_solving_CPLEX(new_flight_legs_covered):
    """
    Using the flight_legs, the leg_to_index_mapper, and index_to_leg_mapper, arrival and departure flights, adjacency matrix of connections
    ,create duties and then create 1 day pairings, 2 day pairings and then create coverage matrix. All these components
    are needed for CPLEX Solving
    """
    flight_legs_chosen=new_flight_legs_covered.copy()
    leg_to_index_mapper=create_leg_to_index_mapper(flight_legs_chosen)
    index_to_leg_mapper=create_index_to_leg_mapper(flight_legs_chosen)
    all_airports,airport_arr_flights,airport_dep_flights=return_all_airports_dep_arr(flight_legs_chosen)
    adjacency_matrix_of_connections=create_adjacency_matrix(flight_legs_chosen)
    dg=generate_duties(all_airports,airport_dep_flights,adjacency_matrix_of_connections,leg_to_index_mapper,flight_legs_chosen)
    pg=PairingsGenerator()
    pg.generate_2_day_pairings(dg.duty_list)
    pg.generate_1_day_pairings(dg.duty_list)
    coverage=coverage_checker(pg,leg_to_index_mapper,flight_legs_chosen)
    return flight_legs_chosen,leg_to_index_mapper,index_to_leg_mapper,all_airports,airport_arr_flights,airport_dep_flights,adjacency_matrix_of_connections,dg,pg,coverage

In [ ]:
def uncov_airports_not_in_cb(flight_coverage_status,legs):
    """
    For the purpose of printing the status, in each divide and cover iteration, this function is used.
    It basically takes all legs having a False indicator in the flight_coverage_status and 
    then checks if the origin is in crew base->if not add it to uncov airports
    and similarly checks for the destination.

    """
    cb=PairingsGenerator().crew_base
    uncov_airports=set()
    for k,v in flight_coverage_status.items():
        if v==False:
            if legs[k].origin not in cb:
                uncov_airports.add(legs[k].origin)
            if legs[k].destination not in cb:
                uncov_airports.add(legs[k].destination)
    return list(set(uncov_airports))
            

In [ ]:
def controller_update_flight_coverage_status(old_cov_status,new_cov_status):
    """
    This function is used in the divide and cover controller function
    Basically, the divide and controller function calls divide and cover function many times.
    In each iteration, new sets of pairings are generated covering different sets of flight indices
    This covering is then updated in old(original)_cov_status. 
    Note in the controller, only if the original flight coverage status all take values True, then only
    the algorithm/While loop is stopped.
    """
    for k,v in new_cov_status.items():
        if v==True:
            old_cov_status[k]=True
    return old_cov_status
def divide_and_cover_controller(K,legs):
    """
    This function controls the divide and cover sub function. It runs many independent iterations of divide and cover
    each of which already execute 300 iterations(or count=main_iter set inside function) so that uncovered flights
    can be quickly covered
    """
    flight_coverage_status=create_flight_coverage_status(legs)
    global_flight_legs_copy=legs.copy()
    updated_flight_legs=legs.copy()
    PG=[]
    pg=PairingsGenerator()
    pg.generate_short_round_trip_pairings(legs)
    flight_coverage_status=init_update_flight_coverage_status(pg.pairings,flight_coverage_status)
    PG=init_update_pairing_list(pg.pairings,PG)
    no_of_div_cover_iterations=10
#     for i in range(no_of_div_cover_iterations):
    while list(flight_coverage_status.values()) != [True]*len(flight_coverage_status.values()):
        new_result=divide_and_cover_new(K,legs)
        new_pairings=new_result[0]
        new_status=new_result[1]
        flight_coverage_status=controller_update_flight_coverage_status(flight_coverage_status,new_status)
        PG=init_update_pairing_list(new_pairings,PG)
        print("Controller @@@@@ Current Status::: True=",list(flight_coverage_status.values()).count(True),"False=",list(flight_coverage_status.values()).count(False))
    return PG

def unique_flights_creator(airports_list,flights_list_by_airport,flt_list=[],flt_indices_list=[]):
    """
    This takes into 2 inputs as params(airports_list:Consists of airports, and flights_list_by_airport:
    Consists of flights by airports).
    
    flt_indices_list:consisting of indices of flights and flt_list:consisting of flights are either passed 
    as parameters or empty lists are used as default parameters
    
    The main role of this function is to remove duplicates
    
    There are many "search reduction heuristics" in divide_and_cover function. In order to optimize this
    flight_leg_chosen space keeps changing. In order to make sure that no duplicates enter this space
    (Again, duplicates cause problems in coverage_matrix generation and then subsequent IP CPLEX Solving),
    this function is implemented
    """

    for airport in airports_list:
        for leg in flights_list_by_airport[airport]:
            if leg.index not in flt_indices_list:
                flt_indices_list.append(leg.index)
                flt_list.append(leg)

    return flt_indices_list,flt_list

def unique_airports_creator(all_airports_list,excluding_airports_list):
    """
    Takes in 2 inputs(all_airports_list and excluding_airports_list:the airports that need to be excluded)
    We form 2 lists, wherein airports that exist in all_airports_list and not existing in excluding_airports_list is,
    appended into focus_airports_list
    and airports that do exist in excluding_airports_list is put in non_focus_airports_list
    """
    focus_airports_list=[]
    non_focus_airports_list=[]
    for airport in all_airports_list:
        if airport not in excluding_airports_list:
            focus_airports_list.append(airport) 
        else:
            non_focus_airports_list.append(airport)
    return focus_airports_list,non_focus_airports_list
            

def unique_flights_creator_random_type(exist_flights_indices,exist_flights_list,K,flights_list):
    """
    This follows the same core concept as unique_flights_creator,
    However, due to the addition of a random_flight_index_generated element, things slightly change while programming and checking
    Input: ()
    This function again, sends a duplicate free list of indices and flights
    """
    iter_=0
    while len(exist_flights_list)!=K and iter_<30:
        r=random.randint(0,len(flights_list)-1)
        if flights_list[r].index not in exist_flights_indices:
            exist_flights_list.append(flights_list[r])
            exist_flights_indices.append(flights_list[r].index)
        iter_+=1
    return exist_flights_list,exist_flights_indices
    

In [ ]:
def divide_and_cover_new(K,legs):
    """
    This is the divide and cover function that executes main_iter no of iterations. It stops either if the main_iter 
    count is reached or if all flights have been covered. Some search heuristics have been deployed if the no. of flights 
    to be covered reduces and try as fast as possible to cover them quickly.
    """
    flight_coverage_status=create_flight_coverage_status(legs)
    global_flight_legs_copy=legs.copy()
    updated_flight_legs=legs.copy()
    
    
    PG=[]
    retry_iter=0
    global_retry_iter=0
    main_iter=0
    
    
    while list(flight_coverage_status.values())!=[True]*len(flight_coverage_status.values()) and main_iter<300:
        if len(updated_flight_legs)<K:
            if list(flight_coverage_status.values()).count(False)>20:
                if global_retry_iter>10:
                    updated_flight_legs=global_flight_legs_copy.copy()
                    flight_legs_chosen=random.sample(updated_flight_legs,K)
                    global_retry_iter=0
                else:
                    flight_legs_chosen=updated_flight_legs.copy()
            else:
                
                if list(flight_coverage_status.values()).count(False)<5:
                    all_airports,airport_arr_flights,airport_dep_flights=return_all_airports_dep_arr(flight_legs_chosen)
                    tpgcb=PairingsGenerator().crew_base
                    crucial_airports=[]
                    non_crucial_airports=[]
                    crucial_airports,non_crucial_airports=unique_airports_creator(all_airports,tpgcb)
                    crucial_airports=list(set(crucial_airports))
                    non_crucial_airports=list(set(non_crucial_airports))
                    slightly_imp_indices,slightly_imp=unique_flights_creator(crucial_airports,airport_arr_flights)
                    slightly_imp_indices,slightly_imp=unique_flights_creator(crucial_airports,airport_dep_flights,slightly_imp,slightly_imp_indices)
                    if len(slightly_imp)<K:
                        if random.random()<0.5:
                            non_crucial_flt_indices,non_crucial_flights=unique_flights_creator(non_crucial_airports,airport_dep_flights)
                            slightly_imp,slightly_imp_indices=unique_flights_creator_random_type(slightly_imp_indices,slightly_imp,K,non_crucial_flights)
                            if len(slightly_imp)<K:
                                slightly_imp,slightly_imp_indices=unique_flights_creator_random_type(slightly_imp_indices,slightly_imp,K,legs)
                        else:
                            if len(slightly_imp)<K:
                                slightly_imp,slightly_imp_indices=unique_flights_creator_random_type(slightly_imp_indices,slightly_imp,K,legs)
                    flight_legs_chosen=slightly_imp.copy()
                    updated_flight_legs=flight_legs_chosen.copy()
                else:
                    imp_flt_indices=[]
                    for k,v in flight_coverage_status.items():
                        if v==False:
                            imp_flt_indices.append(k)
                    flight_legs_chosen=[]
                    for i in imp_flt_indices:
                        flight_legs_chosen.append(legs[i])
                    flight_legs_chosen,imp_flt_indices=unique_flights_creator_random_type(imp_flt_indices,flight_legs_chosen,K,legs)
                    updated_flight_legs=flight_legs_chosen.copy()
                    
            global_retry_iter+=1
        else:
            flight_legs_chosen=random.sample(updated_flight_legs,K)
            
            
        if len(flight_legs_chosen)>K:
            flight_legs_chosen=random.sample(flight_legs_chosen,K)
            
        if global_retry_iter>20:
            global_retry_iter=0
            flight_legs_chosen=random.sample(global_flight_legs_copy,K)
        flight_legs_chosen,leg_to_index_mapper,index_to_leg_mapper,all_airports,airport_arr_flights,airport_dep_flights,adjacency_matrix_of_connections,dg,pg,coverage=init_for_solving_CPLEX(flight_legs_chosen.copy())
        
        if coverage[0]==False or len(pg.pairings)<2:
            while coverage[0]==False:
                if len(flight_legs_chosen)==0:
                    break
                new_flight_legs_covered,new_coverage_matrix=new_coverage_matrix_and_legs(coverage,flight_legs_chosen.copy())
                flight_legs_chosen,leg_to_index_mapper,index_to_leg_mapper,all_airports,airport_arr_flights,airport_dep_flights,adjacency_matrix_of_connections,dg,pg,coverage=init_for_solving_CPLEX(new_flight_legs_covered.copy())
                  
            sol,airline_cp_model,x=create_and_solve_model(pg,leg_to_index_mapper,flight_legs_chosen,IP=True)
            
            if sol is None:
                continue
            else:
                pairing_solution_indices=return_pairing_information(pg,airline_cp_model,x).copy()
                PG=update_pairing_set(PG,pg,pairing_solution_indices,legs,index_to_leg_mapper,leg_to_index_mapper).copy()
                flight_coverage_status=update_flight_coverage_status(flight_coverage_status,flight_legs_chosen)
                
        else:
            sol,airline_cp_model,x=create_and_solve_model(pg,leg_to_index_mapper,flight_legs_chosen,IP=True)
            if sol is None:
                continue
            else:
                pairing_solution_indices=return_pairing_information(pg,airline_cp_model,x).copy()
                PG=update_pairing_set(PG,pg,pairing_solution_indices,legs,index_to_leg_mapper,leg_to_index_mapper).copy()
                flight_coverage_status=update_flight_coverage_status(flight_coverage_status,flight_legs_chosen)
        
        updated_flight_legs=[]
        if main_iter%10==0:
            updated_flight_legs=update_flight_list(updated_flight_legs,legs,global_flight_legs_copy,flight_coverage_status,K,main_iter)
        

        print("Current Status::: True=",list(flight_coverage_status.values()).count(True),"False=",list(flight_coverage_status.values()).count(False),"RETRY ITER",retry_iter,"GLOBALRETRYITER",global_retry_iter,"MAIN ITER",main_iter)
        print("UNCOVERED AIRPORT :::: ",uncov_airports_not_in_cb(flight_coverage_status,legs))
        main_iter+=1

    return (PG,flight_coverage_status)


In [ ]:
#df=pd.read_csv(r"C:\Users\Acer Pc\Desktop\UltimateAir\UltimateAirUpdatedPBIX.csv")
df=pd.read_csv(r"Indigo_Schedule.csv")
legs=create_legs_object(df,days=2)
PG=divide_and_cover_controller(200,legs)

In [ ]:
import random
def CG_stopper(CG_tracker):
    """
    If False is returned by CG_stopper, then continue CG iterations
    Else if True returned, stop!
    """
    if len(CG_tracker)==0 or len(CG_tracker)==1:
        return False
    else:
        if len(CG_tracker[-2][0].pairings)-len(CG_tracker[-1][0].pairings) == 0:
            # No pairings added
            return True
        else:
            return False

def get_duties_reduced_costs(duty_list):
    temp=[]
    for duty in duty_list:
        temp.append(duty.duty_reduced_cost)
    return temp.copy()
def column_generation(PG,dg,duty_network,crew_bases_indices,s_nodes_dict,t_nodes_dict):
    """
    Each step is briefly highlighted in the below lines.
    Overall, column generation keeps running and adds columns by calling SPPRC until a negative reduced cost column 
    is not available for addition.
    Do note that a PG list enters into column_generation as a parameter. PG contains the divide_and_cover related 
    distribution of pairings that covers all flights
    
    """
    #1. Starting off
    #df=pd.read_csv(r"C:\Users\Acer Pc\Desktop\UltimateAir\UltimateAirUpdatedPBIX.csv")
    df=pd.read_csv(r"Indigo_Schedule.csv")
    legs=create_legs_object(df,days=2)
    legs=legs
    all_airports,airport_arr_flights,airport_dep_flights=return_all_airports_dep_arr(legs)
    adjacency_matrix_of_connections=create_adjacency_matrix(legs)
    leg_to_index_mapper=create_leg_to_index_mapper(legs)
    index_to_leg_mapper=create_index_to_leg_mapper(legs)
    
    
    #2. Performing Divide and Cover Heuristic->This would have been performed outside of CG and fed into this method
#     PG=divide_and_cover_new(300,legs,[])
#     print(len(PG))
#     return PG,legs
    pg=PairingsGenerator(PG)
    CG_tracker=[]
    #RCT Stands for Reduced costs tracker
    RCT=[]
#     return pg,legs
    
    print("Duty Generation Started")
    #3. Generating Duty List
#     dg=generate_duties(all_airports,airport_dep_flights,adjacency_matrix_of_connections,leg_to_index_mapper,legs)
    print("DutyGeneration Completed")
    
    #--Here we are setting termination criteria to 10 iterations
    no_of_iterations=10
    
    #4. Form Duty Network
    crew_bases=pg.crew_base
    print("Duty Network Creation Started")
    #duty_network,crew_bases_indices,s_nodes_dict,t_nodes_dict=form_duty_network(dg,crew_bases)
    print("Duty Network Creation Ended")
    
    #5. Column Generation Procedure
    print("CG Started")
    while CG_stopper(CG_tracker)!=True:
        sol,airline_cp_model,x=create_and_solve_model(pg,leg_to_index_mapper,legs)
        print(sol.get_objective_value())
        existing_pg_list=pg.pairings.copy()
        pairing_solution_indices,dual_values=default_model_return_function(sol,airline_cp_model,x,pg)
        CG_tracker.append((pg,sol,airline_cp_model,x,pairing_solution_indices.copy(),dual_values.copy(),dg.duty_list.copy()))
        RCT.append(get_duties_reduced_costs(dg.duty_list).copy())
        dg=modify_duty_costs(dual_values,dg)
        for airport_letter_code,airport_tuple in crew_bases_indices.items():
            labels=SPPRC(airport_letter_code,duty_network,crew_bases_indices,s_nodes_dict,t_nodes_dict,dg)
            existing_pg_list=process_labels_and_collect_pairings(existing_pg_list,labels,dual_values).copy()
            pg=PairingsGenerator(existing_pg_list.copy())
    return (CG_tracker,RCT)

            

In [ ]:
df=pd.read_csv(r"Indigo_Schedule.csv")
legs=create_legs_object(df,days=2)
legs=legs
all_airports,airport_arr_flights,airport_dep_flights=return_all_airports_dep_arr(legs)
adjacency_matrix_of_connections=create_adjacency_matrix(legs)
leg_to_index_mapper=create_leg_to_index_mapper(legs)
index_to_leg_mapper=create_index_to_leg_mapper(legs)
dg=generate_duties(all_airports,airport_dep_flights,adjacency_matrix_of_connections,leg_to_index_mapper,legs)
pg=PairingsGenerator(PG)
crew_bases=pg.crew_base
duty_network,crew_bases_indices,s_nodes_dict,t_nodes_dict=form_duty_network(dg,crew_bases)
with open("DutyNetwork.txt","w") as f:
    f.write(str(duty_network))

In [ ]:
PG_copy=PG.copy()

In [ ]:
duty_network=list(duty_network)

In [ ]:
for i in range(len(duty_network)):
    duty_network[i]=np.array(duty_network[i])
print(duty_network[0])

In [ ]:
with open("DutyNetwork.txt","w") as f:
    f.write(str(duty_network))

In [ ]:
CG_tracker=column_generation(PG,dg,duty_network,crew_bases_indices,s_nodes_dict,t_nodes_dict)


In [ ]:
l1=[[1,2,3,4],[1,2,3,4]]
with open("Temp.txt","w") as f:
    f.write(str(l1))

In [ ]:
z=coverage_checker(PairingsGenerator(PG),create_leg_to_index_mapper(legs),legs)

In [ ]:
f=[]
for item in CG_tracker[-1]:
    f.append(item)
data=[]
for item in f:
    data.append([])
    for d in item:
        data[-1].append(d)
import numpy as np
data=list(np.array(data).T)

In [ ]:
from tabulate import tabulate

def print_table(data):
    if not data:
        print("The data is empty.")
    else:
        headers = ["Column 1", "Column 2", "Column 3", "Column 4"]  # Adjust column headers as needed

        # Use the "tabulate" function to format and print the table
        print(tabulate(data, headers=headers, tablefmt="pretty"))


print_table(data)




In [ ]:
!pip install tabulate

In [ ]:
c=[1,2]
class ABC:
    def __init__(self,l):
        self.l=l
a=ABC(c)
b=ABC(c)


In [ ]:
leg_to_index_mapper=create_leg_to_index_mapper(legs)

In [ ]:
leg_to_index_mapper[179]